In [0]:
import pandas as pd
import numpy as np
import urllib, tarfile, math
import urllib.request

In [0]:
# create a new folder to store extracted gzip. Not used in this code.
# dbutils.fs.mkdirs("/tmp/unpack")

### Downloading the dataset with gzip compression

In [0]:
# Download tar.gz (543MB)
url = "http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz"
filename, headers = urllib.request.urlretrieve(url, "/tmp/lastfm-dataset-360K.tar.gz") 

In [0]:
filename, headers

Out[10]: ('/tmp/lastfm-dataset-360K.tar.gz',
 <http.client.HTTPMessage at 0x7f8d6cf130d0>)

#### Unzipping the tar gz file to unload contents into the Filestore

In [0]:
%sh
tar xvzf /tmp/lastfm-dataset-360K.tar.gz -C /tmp/

lastfm-dataset-360K/
lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv
lastfm-dataset-360K/README.txt
lastfm-dataset-360K/mbox_sha1sum.py
lastfm-dataset-360K/usersha1-profile.tsv

### Checking the file directory for unzipped files

In [0]:
dbutils.fs.ls('file:/tmp/lastfm-dataset-360K/')

Out[28]: [FileInfo(path='file:/tmp/lastfm-dataset-360K/usersha1-profile.tsv', name='usersha1-profile.tsv', size=24644914),
 FileInfo(path='file:/tmp/lastfm-dataset-360K/mbox_sha1sum.py', name='mbox_sha1sum.py', size=199)]

In [0]:
# copy each extracted file one at a time
#dbutils.fs.mv("file:/tmp/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv", "dbfs:/FileStore/tables/lastfm-data/usersha1-artmbid-artname-plays.tsv")

Out[25]: True

### Copying the uncompressed files from temp dir to Databricks Filestore

In [0]:
# copy all files at once
dbutils.fs.mv("file:/tmp/lastfm-dataset-360K/", "dbfs:/FileStore/tables/lastfm-data/", recurse=True)

Out[30]: True

### Following code converts the tsv to Hive table

The file is read as a dataframe, columns are renamed and saved as SQL table

In [0]:
# File location and type
file_location = 'dbfs:/FileStore/tables/lastfm-data/usersha1-artmbid-artname-plays.tsv'
file_type = "csv"
# CSV options
infer_schema = "true"
first_row_is_header = "false"
delimiter = "\t"
# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.withColumnRenamed("_c0", "userid")\
       .withColumnRenamed("_c1", "artistid")\
       .withColumnRenamed("_c2", "artistname")\
       .withColumnRenamed("_c3", "plays")\

display(df)
# Create a view or table
temp_table_name = "plays"
df.createOrReplaceTempView(temp_table_name)

permanent_table_name = "plays"
df.write.format("delta").saveAsTable(permanent_table_name)

In [0]:
%sql

CREATE TABLE IF NOT EXISTS plays
  LOCATION 'dbfs:/FileStore/tables/lastfm-data/plays'

In [0]:
%sql
/* Query the created temp table in a SQL cell */
select * from `plays`
